# Computation

In [ ]:
# csir_alignment_eval_aggregates_only.py
# pip install sentence-transformers torch pandas numpy tqdm xlsxwriter

import os, json, math, re, gc
from pathlib import Path
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass

import numpy as np
import pandas as pd
import torch
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer

# ============== Config ==============
# Paths (override with env vars if needed)
QUERIES_PATH = os.getenv("QUERIES_PATH", "../Data/queries_subset.jsonl")
DOCS_PATH    = os.getenv("DOCS_PATH",    "../Data/docs_subset.jsonl")
RELS_PATH    = os.getenv("RELS_PATH",    "../Data/qrels_subset.jsonl")

# Output roots
RUN_ROOT   = Path("./alignment_eval_runs")         # per-(model, doc-lang) aggregate CSVs
AGG_ROOT   = Path("./alignment_eval_aggregates")   # global merged aggregate CSVs
CACHE_ROOT = Path("./alignment_eval_cache")        # cached embeddings
EXCEL_PATH = Path("./alignment_eval_results.xlsx") # single Excel with aggregate tables

for p in (RUN_ROOT, AGG_ROOT, CACHE_ROOT):
    p.mkdir(parents=True, exist_ok=True)

# Models (Hugging Face IDs) — EMs only
MODELS = [
    "Alibaba-NLP/gte-multilingual-base",
    "intfloat/multilingual-e5-small",
    "intfloat/multilingual-e5-base",
    "intfloat/multilingual-e5-large",
    "jinaai/jina-embeddings-v3",
    "upskyy/bge-m3-korean",
]

# Retrieval & eval
TOPK = int(os.getenv("TOPK", "10"))
BATCH_SIZE = int(os.getenv("BATCH_SIZE", "256"))
SWITCH_BINS = [(0.0,0.2),(0.2,0.4),(0.4,0.6),(0.6,0.8),(0.8,1.0)]  # closed-left

# CUDA settings
HAS_CUDA = torch.cuda.is_available()
DEVICE = "cuda" if HAS_CUDA else "cpu"

# ============== Utilities ==============
def safe_model_slug(model_id: str) -> str:
    return re.sub(r"[^a-zA-Z0-9_.-]+", "_", model_id.replace("/", "_"))

def read_jsonl(path: str) -> List[dict]:
    out = []
    with open(path, "r", encoding="utf-8") as f:
        for ln in f:
            ln = ln.strip()
            if ln:
                out.append(json.loads(ln))
    return out

def load_data(q_path=QUERIES_PATH, d_path=DOCS_PATH, r_path=RELS_PATH):
    queries = read_jsonl(q_path)
    docs = read_jsonl(d_path)
    rels = read_jsonl(r_path)
    q_by_id = {str(q.get("id", q.get("query_id"))): q for q in queries}
    d_by_id = {str(d.get("doc_id")): d for d in docs}
    rel_pos: Dict[str, set] = {}
    for r in rels:
        if r.get("relevance", 0) > 0:
            rel_pos.setdefault(str(r["query_id"]), set()).add(str(r["doc_id"]))
    return queries, docs, rels, q_by_id, d_by_id, rel_pos

def _norm_lang_alias(x: str) -> str:
    x = (x or "").lower()
    if x.startswith(("es","spa")): return "es"
    if x.startswith(("en","eng")): return "en"
    if x.startswith(("zh","chi","zho","cn")): return "zh"
    return x

def _candidate_keys(obj_keys, lang_key, is_query):
    aliases = {
        "en": ["en","eng","english"],
        "es": ["es","spa","spanish"],
        "zh": ["zh","chi","zho","cn","zh_cn","zh-cn","chinese"],
    }.get(lang_key, [lang_key])
    cands = []
    if is_query:
        for a in aliases:
            cands.append(f"{a}_q")
    for a in aliases:
        cands.extend([a, f"{a}_doc", f"{a}_d"])
    if is_query:
        pat = re.compile(rf".*(?:^|_)(?:{'|'.join(map(re.escape, aliases))})(?:_|$).*_q$")
        for k in obj_keys:
            if pat.match(k):
                cands.append(k)
    # dedupe keep order
    seen, out = set(), []
    for k in cands:
        if k not in seen:
            seen.add(k); out.append(k)
    return out

def _get_text_value(v):
    if isinstance(v, str):
        return v.strip()
    if isinstance(v, dict):
        t = (v.get("text") or v.get("value") or "").strip()
        if t:
            return t
    return ""

def texts_for_lang(items: List[dict], ids: List[str], lang_key: str, is_query: bool) -> List[str]:
    id_key = "query_id" if is_query else "doc_id"
    d = {str(it.get(id_key, it.get("id"))): it for it in items}
    out, empties = [], []
    for id_ in ids:
        obj = d[id_]
        val = ""
        if lang_key in obj:
            val = _get_text_value(obj[lang_key])
        if not val:
            for ck in _candidate_keys(obj.keys(), lang_key, is_query):
                if ck in obj:
                    val = _get_text_value(obj[ck]); 
                    if val: break
        if is_query and not val:
            for k,v in obj.items():
                if isinstance(k, str) and k.endswith("_q"):
                    val = _get_text_value(v)
                    if val: break
        if not val:
            empties.append(id_); val=""
        out.append(val)
    if empties:
        print(f"[WARN] {len(empties)} empty for lang='{lang_key}' is_query={is_query}. ex={empties[:5]}")
    return out

def build_sw_rate_lookup(queries: List[dict]) -> Dict[Tuple[str, str], float]:
    out: Dict[Tuple[str,str], float] = {}
    for o in queries:
        qid = str(o.get("id") or o.get("query_id") or "").strip()
        if not qid: continue
        for pair in ("es_en","zh_en","zh_es"):
            q_key = f"{pair}_q"
            sw_key = f"{pair}_sw_rate"
            if q_key in o and isinstance(o[q_key], str) and o[q_key].strip():
                val = o.get(sw_key, None)
                if val is not None:
                    try:
                        out[(q_key, qid)] = float(val)
                    except Exception:
                        pass
    return out

# ============== Math helpers ==============
def unit(x: np.ndarray) -> np.ndarray:
    n = np.linalg.norm(x, axis=-1, keepdims=True) + 1e-12
    return x / n

def cos(a: np.ndarray, b: np.ndarray) -> float:
    a, b = unit(a), unit(b)
    return float(np.clip((a * b).sum(-1), -1.0, 1.0))

def angle(a: np.ndarray, b: np.ndarray) -> float:
    return float(math.acos(np.clip(cos(a,b), -1.0, 1.0)))  # radians

def centroid(vecs: np.ndarray) -> np.ndarray:
    if vecs.size == 0:
        return vecs
    return unit(unit(vecs).mean(axis=0))

@torch.inference_mode()
def topk_indices(Q: np.ndarray, D: np.ndarray, k: int) -> np.ndarray:
    dev = "cuda" if HAS_CUDA else "cpu"
    dtype = torch.float16 if HAS_CUDA else torch.float32
    Qt = torch.from_numpy(Q).to(dev, dtype=dtype, non_blocking=True)
    Dt = torch.from_numpy(D).to(dev, dtype=dtype, non_blocking=True)
    sims = (Qt @ Dt.T).to(torch.float32)
    k = min(k, Dt.shape[0])
    _, idx = torch.topk(sims, k=k, largest=True, sorted=True, dim=1)
    out = idx.detach().cpu().numpy()
    del Qt, Dt, sims
    if HAS_CUDA: torch.cuda.empty_cache()
    return out

# ============== Embedding (cached) ==============
@dataclass
class EncCfg:
    model_id: str
    batch_size: int = BATCH_SIZE

def _cache_path(model_id: str, kind: str, lang: str) -> Path:
    safe = safe_model_slug(model_id)
    return CACHE_ROOT / f"{safe}__{kind}_{lang}.npy"

def load_model(model_id: str) -> SentenceTransformer:
    m = SentenceTransformer(model_id, device=DEVICE, trust_remote_code=True)
    return m

def encode_cached(model: SentenceTransformer, model_id: str, kind: str, lang: str, texts: List[str]) -> np.ndarray:
    path = _cache_path(model_id, kind, lang)
    if path.exists():
        return np.load(path)
    embs = model.encode(
        texts,
        batch_size=BATCH_SIZE,
        convert_to_numpy=True,
        normalize_embeddings=True,
        show_progress_bar=True,
    ).astype(np.float32)
    np.save(path, embs)
    return embs

# ============== Grouping / Binning / Accumulators ==============
Q_LANG_KEYS = ["en_q","es_q","zh_q","es_en_q","zh_en_q","zh_es_q"]
DOC_LANGS   = ["en","es","zh"]

def primary_lang_of(q_lang_key: str) -> str:
    key = q_lang_key[:-2] if q_lang_key.endswith("_q") else q_lang_key
    first = key.split("_")[0]
    return _norm_lang_alias(first)

def sw_bin_label(sw: Optional[float]) -> str:
    if sw is None or (isinstance(sw, float) and (math.isnan(sw))):
        return "NA"
    x = float(sw)
    for lo, hi in SWITCH_BINS:
        # include exact 1.0 in the last bin
        if (lo <= x < hi) or (math.isclose(x,1.0) and math.isclose(hi,1.0)):
            return f"{lo:.1f}-{hi:.1f}"
    return "NA"

class MeanAccumulator:
    """Accumulate sums and counts per group key; emit means."""
    def __init__(self, fields: List[str]):
        self.fields = fields
        self.sums: Dict[Tuple, List[float]] = {}
        self.counts: Dict[Tuple, int] = {}

    def add(self, key: Tuple, values: Dict[str, float]):
        if key not in self.sums:
            self.sums[key] = [0.0]*len(self.fields)
            self.counts[key] = 0
        row = self.sums[key]
        for i, f in enumerate(self.fields):
            row[i] += float(values.get(f, 0.0))
        self.counts[key] += 1

    def to_dataframe(self, columns_prefix: Optional[List[str]] = None):
        records = []
        for key, sums in self.sums.items():
            cnt = self.counts[key]
            means = [ (v / cnt if cnt>0 else float('nan')) for v in sums ]
            rec = dict()
            # unpack key into group columns
            if columns_prefix:
                for name, val in zip(columns_prefix, key):
                    rec[name] = val
            else:
                # unnamed tuple -> generic g1,g2...
                for i, val in enumerate(key):
                    rec[f"g{i+1}"] = val
            for f, m in zip(self.fields, means):
                rec[f] = m
            rec["N"] = cnt
            records.append(rec)
        return pd.DataFrame(records)

# ---------- Resumability helpers ----------
def model_run_dir(model_id: str) -> Path:
    return RUN_ROOT / safe_model_slug(model_id)

def out_csv_path(model_id: str, d_lang: str, kind: str) -> Path:
    # kind in {"QQ","QD","Jaccard"}
    return model_run_dir(model_id) / f"D_{d_lang}_{kind}.csv"

def done_flag_path(model_id: str, d_lang: str) -> Path:
    return model_run_dir(model_id) / f"D_{d_lang}.done"

def mark_done(model_id: str, d_lang: str):
    f = done_flag_path(model_id, d_lang)
    f.parent.mkdir(parents=True, exist_ok=True)
    f.write_text("ok", encoding="utf-8")

def is_done(model_id: str, d_lang: str) -> bool:
    return done_flag_path(model_id, d_lang).exists()

# ---------- Excel builder (merge aggregate CSVs) ----------
def rebuild_excel_and_global_csvs():
    qq_list, qd_list, j_list = [], [], []
    for mdir in RUN_ROOT.glob("*"):
        if not mdir.is_dir(): continue
        for f in mdir.glob("D_*_QQ.csv"):
            try: qq_list.append(pd.read_csv(f))
            except Exception: pass
        for f in mdir.glob("D_*_QD.csv"):
            try: qd_list.append(pd.read_csv(f))
            except Exception: pass
        for f in mdir.glob("D_*_Jaccard.csv"):
            try: j_list.append(pd.read_csv(f))
            except Exception: pass

    AGG_ROOT.mkdir(parents=True, exist_ok=True)

    agg_qq = pd.concat(qq_list, ignore_index=True) if qq_list else pd.DataFrame()
    agg_qd = pd.concat(qd_list, ignore_index=True) if qd_list else pd.DataFrame()
    agg_j  = pd.concat(j_list,  ignore_index=True) if j_list  else pd.DataFrame()

    if not agg_qq.empty: (AGG_ROOT / "agg_query_query.csv").write_text(agg_qq.to_csv(index=False), encoding="utf-8")
    if not agg_qd.empty: (AGG_ROOT / "agg_query_docs.csv").write_text(agg_qd.to_csv(index=False), encoding="utf-8")
    if not agg_j.empty:  (AGG_ROOT / "agg_jaccard.csv").write_text(agg_j.to_csv(index=False),  encoding="utf-8")

    with pd.ExcelWriter(EXCEL_PATH, engine="xlsxwriter") as writer:
        if not agg_qq.empty: agg_qq.to_excel(writer, sheet_name="QQ Aggregates", index=False)
        if not agg_qd.empty: agg_qd.to_excel(writer, sheet_name="QD Aggregates", index=False)
        if not agg_j.empty:  agg_j.to_excel(writer, sheet_name="Jaccard Aggregates", index=False)

        # Optional pivots
        try:
            if not agg_qq.empty:
                pivot_eas = (agg_qq.groupby(["model","doc_lang","query_lang_cs"])
                                   [["EAS"]].mean().reset_index())
                pivot_eas.to_excel(writer, sheet_name="Pivot EAS", index=False)
            if not agg_qd.empty:
                pivot_cd = (agg_qd.groupby(["model","doc_lang","query_lang_cs"])
                                   [["CentroidDrift@K"]].mean().reset_index())
                pivot_cd.to_excel(writer, sheet_name="Pivot CentroidDrift", index=False)
        except Exception:
            pass

    print(f"[Excel] Updated workbook → {EXCEL_PATH}")

# ============== Main ==============
def main():
    print("Loading data...")
    queries, docs, rels, q_by_id, d_by_id, rel_pos = load_data()
    qids = [str(q.get("id", q.get("query_id"))) for q in queries]
    dids = [str(d.get("doc_id")) for d in docs]

    # Pre-extract texts per language
    DOC_LANGS = ["en","es","zh"]
    Q_LANG_KEYS = ["en_q","es_q","zh_q","es_en_q","zh_en_q","zh_es_q"]
    docs_text_cache   = {dl: texts_for_lang(docs, dids, dl, is_query=False) for dl in DOC_LANGS}
    queries_text_cache= {ql: texts_for_lang(queries, qids, ql, is_query=True) for ql in Q_LANG_KEYS}
    sw_rate_map = build_sw_rate_lookup(queries)

    doc_id_arr = np.array(dids, dtype=object)

    for model_id in MODELS:
        print(f"\n=== Model: {model_id} ===")
        mdir = model_run_dir(model_id)
        mdir.mkdir(parents=True, exist_ok=True)

        try:
            model = load_model(model_id)

            # Cache embeddings per language
            D_by_lang: Dict[str, np.ndarray] = {}
            for d_lang in DOC_LANGS:
                D_by_lang[d_lang] = encode_cached(model, model_id, "docs", d_lang, docs_text_cache[d_lang])

            Q_by_lang: Dict[str, np.ndarray] = {}
            for q_lang in Q_LANG_KEYS:
                Q_by_lang[q_lang] = encode_cached(model, model_id, "queries", q_lang, queries_text_cache[q_lang])

            # Per doc language processing (resumable)
            for d_lang in DOC_LANGS:
                if is_done(model_id, d_lang):
                    print(f"[RESUME] {model_id} | D={d_lang}: already completed. Skipping.")
                    continue

                print(f"[RUN] {model_id} | D={d_lang}: computing aggregates…")

                try:
                    D = D_by_lang[d_lang]

                    # Precompute TopK indices for all q_lang against this D
                    topk_by_q: Dict[str, np.ndarray] = {}
                    for q_lang in Q_by_lang.keys():
                        topk_by_q[q_lang] = topk_indices(Q_by_lang[q_lang], D, TOPK)

                    # Accumulators
                    acc_QQ = MeanAccumulator(fields=["EAS","angle_rad","qq_euclid"])
                    acc_QD = MeanAccumulator(fields=["ΔQC@K","CentroidDrift@K","AlignDrift@K"])
                    acc_J  = MeanAccumulator(fields=["Jaccard@K"])

                    # --- Query–Query & Query–Docs alignment ---
                    for q_lang_cs in ["es_en_q","zh_en_q","zh_es_q"]:
                        q_lang_mono = primary_lang_of(q_lang_cs) + "_q"
                        Q_cs   = Q_by_lang[q_lang_cs]
                        Q_mono = Q_by_lang[q_lang_mono]
                        T_cs   = topk_by_q[q_lang_cs]
                        T_mono = topk_by_q[q_lang_mono]

                        for i, qid in enumerate(qids):
                            q_cs   = Q_cs[i]
                            q_m    = Q_mono[i]
                            EAS    = cos(q_cs, q_m)
                            ang    = angle(q_cs, q_m)
                            qqL2   = float(np.linalg.norm(unit(q_cs) - unit(q_m)))

                            idx_m = T_mono[i]
                            idx_c = T_cs[i]
                            C_m   = centroid(D[idx_m])
                            C_c   = centroid(D[idx_c])

                            QC_m  = cos(q_m, C_m)
                            QC_c  = cos(q_cs, C_c)
                            d_QC  = QC_c - QC_m
                            c_drift = float(np.linalg.norm(unit(C_c) - unit(C_m)))
                            q_drift = unit(q_cs) - unit(q_m)
                            n_drift = unit(C_c)  - unit(C_m)
                            align_drift = cos(q_drift, n_drift) if (np.linalg.norm(q_drift)>0 and np.linalg.norm(n_drift)>0) else 0.0

                            sw = sw_rate_map.get((q_lang_cs, qid), np.nan)
                            swb = sw_bin_label(sw)

                            # Group keys
                            key_QQ = (model_id, d_lang, primary_lang_of(q_lang_cs), q_lang_cs.replace("_","-").replace("-q",""), swb)
                            key_QD = (model_id, d_lang, q_lang_cs.replace("_","-").replace("-q",""), swb)

                            acc_QQ.add(key_QQ, {"EAS":EAS, "angle_rad":ang, "qq_euclid":qqL2})
                            acc_QD.add(key_QD, {"ΔQC@K":d_QC, "CentroidDrift@K":c_drift, "AlignDrift@K":align_drift})

                    # --- Jaccard@K (set overlap) ---
                    for base, cs in [("en_q","es_en_q"), ("en_q","zh_en_q"),
                                     ("es_q","es_en_q"), ("zh_q","zh_en_q"),
                                     ("zh_q","zh_es_q"), ("es_q","zh_es_q")]:
                        T_b = topk_by_q[base]
                        T_c = topk_by_q[cs]
                        base_lbl = base.replace("_","-").replace("-q","")
                        cs_lbl   = cs.replace("_","-").replace("-q","")
                        for i, qid in enumerate(qids):
                            set_b = set(doc_id_arr[T_b[i]])
                            set_c = set(doc_id_arr[T_c[i]])
                            t = len(set_b & set_c)
                            j = t / (2*TOPK - t) if TOPK > 0 else 0.0
                            sw = sw_rate_map.get((cs, qid), np.nan)
                            swb = sw_bin_label(sw)
                            key_J = (model_id, d_lang, base_lbl, cs_lbl, swb)
                            acc_J.add(key_J, {"Jaccard@K": j})

                    # ---- Write (overwrite) per-(model,doc-lang) aggregate CSVs
                    qq_df = acc_QQ.to_dataframe(columns_prefix=["model","doc_lang","query_lang_mono","query_lang_cs","sw_bin"])
                    qd_df = acc_QD.to_dataframe(columns_prefix=["model","doc_lang","query_lang_cs","sw_bin"])
                    j_df  = acc_J.to_dataframe(columns_prefix=["model","doc_lang","query_lang_mono","query_lang_cs","sw_bin"])

                    out_qq = out_csv_path(model_id, d_lang, "QQ")
                    out_qd = out_csv_path(model_id, d_lang, "QD")
                    out_j  = out_csv_path(model_id, d_lang, "Jaccard")

                    qq_df.to_csv(out_qq, index=False, encoding="utf-8")
                    qd_df.to_csv(out_qd, index=False, encoding="utf-8")
                    j_df.to_csv(out_j,  index=False, encoding="utf-8")

                    mark_done(model_id, d_lang)
                    print(f"[DONE] {model_id} | D={d_lang}: wrote {out_qq.name}, {out_qd.name}, {out_j.name}")

                except Exception as e:
                    print(f"[ERROR] {model_id} | D={d_lang}: {e}. Keeping partial results; continuing.")
                    gc.collect()
                    if HAS_CUDA: torch.cuda.empty_cache()
                    continue

        except Exception as e:
            print(f"[ERROR] model load/run failed for {model_id}: {e}. Continuing with next model.")
        finally:
            try:
                del model
            except Exception:
                pass
            gc.collect()
            if HAS_CUDA: torch.cuda.empty_cache()

    # Rebuild global CSVs + Excel (aggregates only)
    rebuild_excel_and_global_csvs()
    print("All done (aggregates only, resumable).")

if __name__ == "__main__":
    main()


Loading data...

=== Model: Alibaba-NLP/gte-multilingual-base ===


Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[RESUME] Alibaba-NLP/gte-multilingual-base | D=en: already completed. Skipping.
[RESUME] Alibaba-NLP/gte-multilingual-base | D=es: already completed. Skipping.
[RESUME] Alibaba-NLP/gte-multilingual-base | D=zh: already completed. Skipping.

=== Model: intfloat/multilingual-e5-small ===
[RESUME] intfloat/multilingual-e5-small | D=en: already completed. Skipping.
[RESUME] intfloat/multilingual-e5-small | D=es: already completed. Skipping.
[RESUME] intfloat/multilingual-e5-small | D=zh: already completed. Skipping.

=== Model: intfloat/multilingual-e5-base ===
[RESUME] intfloat/multilingual-e5-base | D=en: already completed. Skipping.
[RESUME] intfloat/multilingual-e5-base | D=es: already completed. Skipping.
[RESUME] intfloat/multilingual-e5-base | D=zh: already completed. Skipping.

=== Model: intfloat/multilingual-e5-large ===
[RESUME] intfloat/multilingual-e5-large | D=en: already completed. Skipping.
[RESUME] intfloat/multilingual-e5-large | D=es: already completed. Skipping.
[RESUME

flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn i

[RESUME] jinaai/jina-embeddings-v3 | D=en: already completed. Skipping.
[RESUME] jinaai/jina-embeddings-v3 | D=es: already completed. Skipping.
[RESUME] jinaai/jina-embeddings-v3 | D=zh: already completed. Skipping.

=== Model: upskyy/bge-m3-korean ===


...

Batches:   0%|          | 0/40 [00:00<?, ?it/s]

Batches:   0%|          | 0/40 [00:00<?, ?it/s]

Batches:   0%|          | 0/40 [00:00<?, ?it/s]

Batches:   0%|          | 0/40 [00:00<?, ?it/s]

[RUN] upskyy/bge-m3-korean | D=en: computing aggregates…
[DONE] upskyy/bge-m3-korean | D=en: wrote D_en_QQ.csv, D_en_QD.csv, D_en_Jaccard.csv
[RUN] upskyy/bge-m3-korean | D=es: computing aggregates…
[DONE] upskyy/bge-m3-korean | D=es: wrote D_es_QQ.csv, D_es_QD.csv, D_es_Jaccard.csv
[RUN] upskyy/bge-m3-korean | D=zh: computing aggregates…
[DONE] upskyy/bge-m3-korean | D=zh: wrote D_zh_QQ.csv, D_zh_QD.csv, D_zh_Jaccard.csv
[Excel] Updated workbook → alignment_eval_results.xlsx
All done (aggregates only, resumable).


# EXCEL

In [28]:
# build_alignment_report.py
# pip install pandas numpy xlsxwriter

import os
from pathlib import Path
import pandas as pd

# --------- Config ---------
RUN_ROOT   = Path(os.getenv("RUN_ROOT", "./alignment_eval_runs"))   # where per-model CSVs live
EXCEL_PATH = Path(os.getenv("EXCEL_PATH", "./alignment_alignment_report.xlsx"))

# --------- Load helpers ---------
def _load_all(kind: str) -> pd.DataFrame:
    """
    kind in {"QQ","QD"} or "Jaccard" (we don't need Jaccard for these 4 sheets, but kept for completeness).
    Reads all D_*_<kind>.csv from subfolders of RUN_ROOT.
    """
    frames = []
    if not RUN_ROOT.exists():
        return pd.DataFrame()
    for mdir in RUN_ROOT.glob("*"):
        if not mdir.is_dir():
            continue
        for f in mdir.glob(f"D_*_{kind}.csv"):
            try:
                df = pd.read_csv(f)
                frames.append(df)
            except Exception as e:
                print(f"[WARN] failed reading {f}: {e}")
    if frames:
        return pd.concat(frames, ignore_index=True)
    return pd.DataFrame()

def _ensure_cols(df: pd.DataFrame, cols):
    missing = [c for c in cols if c not in df.columns]
    if missing:
        raise ValueError(f"Missing columns in input for this sheet: {missing}")

# --------- Build sheets ---------
def build_sheet_1_qq(df_qq: pd.DataFrame) -> pd.DataFrame:
    """
    Sheet 1: Query–Query results per CS pair (lang1 -> lang2) and model (aggregated across doc_lang and sw_bin)
    Metrics: EAS, angle_rad, qq_euclid, plus N
    """
    need = ["model","query_lang_mono","query_lang_cs","EAS","angle_rad","qq_euclid","N"]
    _ensure_cols(df_qq, need[:-1])  # N may or may not exist; we'll compute
    # Build a 'pair' column like "es→es-en"
    df = df_qq.copy()
    df["pair"] = df["query_lang_mono"].astype(str) + "→" + df["query_lang_cs"].astype(str)
    # If N column exists from accumulator, keep it; otherwise add 1
    if "N" not in df.columns:
        df["N"] = 1
    # Aggregate across doc_lang and sw_bin
    grp = df.groupby(["model","pair"], as_index=False).agg({
        "EAS":"mean",
        "angle_rad":"mean",
        "qq_euclid":"mean",
        "N":"sum"
    })
    # Order columns nicely
    return grp[["model","pair","EAS","angle_rad","qq_euclid","N"]].sort_values(["model","pair"])

def build_sheet_2_qq_bins(df_qq: pd.DataFrame) -> pd.DataFrame:
    """
    Sheet 2: Query–Query results per CS pair & switch-rate bin (and model)
    """
    need = ["model","query_lang_mono","query_lang_cs","sw_bin","EAS","angle_rad","qq_euclid","N"]
    _ensure_cols(df_qq, need[:-1])
    df = df_qq.copy()
    df["pair"] = df["query_lang_mono"].astype(str) + "→" + df["query_lang_cs"].astype(str)
    if "N" not in df.columns:
        df["N"] = 1
    grp = df.groupby(["model","pair","sw_bin"], as_index=False).agg({
        "EAS":"mean",
        "angle_rad":"mean",
        "qq_euclid":"mean",
        "N":"sum"
    })
    # Keep NA bin last by simple sort trick
    grp["sw_bin"] = grp["sw_bin"].fillna("NA").astype(str)
    return grp[["model","pair","sw_bin","EAS","angle_rad","qq_euclid","N"]].sort_values(["model","pair","sw_bin"])

def build_sheet_3_qd(df_qd: pd.DataFrame) -> pd.DataFrame:
    """
    Sheet 3: Query–Docs alignment shift (monolingual vs CS) by model & CS pair (aggregated across doc_lang and sw_bin)
    Metrics: ΔQC@K, CentroidDrift@K, AlignDrift@K, N
    """
    need = ["model","query_lang_cs","ΔQC@K","CentroidDrift@K","AlignDrift@K","N"]
    _ensure_cols(df_qd, need[:-1])
    df = df_qd.copy()
    # For readability, expose a 'pair' = "<mono>→<cs>" using the first tag in cs
    # The mono language is the first token of query_lang_cs (e.g., "es-en" -> "es")
    # but we didn't carry mono explicitly in QD; we keep cs label as the 'pair' for this sheet.
    # If you want explicit mono, uncomment the heuristic below:
    # mono = df['query_lang_cs'].astype(str).str.split('-', n=1).str[0]
    # df["pair"] = mono + "→" + df["query_lang_cs"].astype(str)
    df["pair"] = df["query_lang_cs"].astype(str)

    if "N" not in df.columns:
        df["N"] = 1
    grp = df.groupby(["model","pair"], as_index=False).agg({
        "ΔQC@K":"mean",
        "CentroidDrift@K":"mean",
        "AlignDrift@K":"mean",
        "N":"sum"
    })
    return grp[["model","pair","ΔQC@K","CentroidDrift@K","AlignDrift@K","N"]].sort_values(["model","pair"])

def build_sheet_4_qd_bins(df_qd: pd.DataFrame) -> pd.DataFrame:
    """
    Sheet 4: Query–Docs alignment shift by model, CS pair & sw_bin
    """
    need = ["model","query_lang_cs","sw_bin","ΔQC@K","CentroidDrift@K","AlignDrift@K","N"]
    _ensure_cols(df_qd, need[:-1])
    df = df_qd.copy()
    # See note in sheet 3 about 'pair'; keep cs label
    df["pair"] = df["query_lang_cs"].astype(str)
    if "N" not in df.columns:
        df["N"] = 1
    df["sw_bin"] = df["sw_bin"].fillna("NA").astype(str)
    grp = df.groupby(["model","pair","sw_bin"], as_index=False).agg({
        "ΔQC@K":"mean",
        "CentroidDrift@K":"mean",
        "AlignDrift@K":"mean",
        "N":"sum"
    })
    return grp[["model","pair","sw_bin","ΔQC@K","CentroidDrift@K","AlignDrift@K","N"]].sort_values(["model","pair","sw_bin"])

def main():
    print(f"[Load] Reading aggregates from: {RUN_ROOT}")

    df_qq = _load_all("QQ")
    df_qd = _load_all("QD")

    if df_qq.empty and df_qd.empty:
        raise SystemExit("No aggregate CSVs found under RUN_ROOT. Run your aggregate eval first.")

    # Build sheets
    sheets = {}
    if not df_qq.empty:
        sheets["QQ (by CS pair)"] = build_sheet_1_qq(df_qq)
        sheets["QQ (by CS pair & bin)"] = build_sheet_2_qq_bins(df_qq)
    else:
        print("[Info] No QQ CSVs found; QQ sheets will be omitted.")

    if not df_qd.empty:
        sheets["QD Shift (by CS pair)"] = build_sheet_3_qd(df_qd)
        sheets["QD Shift (by CS pair & bin)"] = build_sheet_4_qd_bins(df_qd)
    else:
        print("[Info] No QD CSVs found; QD sheets will be omitted.")

    # Write Excel
    EXCEL_PATH.parent.mkdir(parents=True, exist_ok=True)
    with pd.ExcelWriter(EXCEL_PATH, engine="xlsxwriter") as writer:
        for name, df in sheets.items():
            # Ensure <= Excel size limits
            max_rows, max_cols = 1_048_576, 16_384
            r, c = df.shape
            if r > max_rows or c > max_cols:
                raise ValueError(f"Sheet '{name}' too large for Excel: {r}x{c}")
            df.to_excel(writer, sheet_name=name[:31], index=False)  # Excel sheet name <=31 chars

    print(f"[OK] Wrote report → {EXCEL_PATH}")
    for name, df in sheets.items():
        print(f"  - {name}: {len(df)} rows")

if __name__ == "__main__":
    main()


[Load] Reading aggregates from: alignment_eval_runs
[OK] Wrote report → alignment_alignment_report.xlsx
  - QQ (by CS pair): 21 rows
  - QQ (by CS pair & bin): 105 rows
  - QD Shift (by CS pair): 21 rows
  - QD Shift (by CS pair & bin): 105 rows
